## Deliverable 2. Create a Customer Travel Destinations Map.

In [12]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [13]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lagoa,PT,39.05,-27.98,64.63,80,90,21.94,light rain
1,1,Wageningen,NL,51.97,5.67,64.00,93,75,11.41,broken clouds
2,2,Ushuaia,AR,-54.80,-68.30,59.00,54,75,4.70,light rain
3,3,Dennis,US,41.74,-70.19,57.99,76,75,6.93,broken clouds
4,4,Barrow,US,71.29,-156.79,26.60,79,90,12.75,snow


In [14]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [15]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
7,7,Kutum,SD,14.20,24.67,80.11,21,53,9.44,broken clouds
9,9,Rikitea,PF,-23.12,-134.97,74.93,84,42,15.64,light rain
11,11,San Lorenzo,PY,-25.33,-57.53,87.80,27,20,6.93,few clouds
15,15,Hilo,US,19.73,-155.09,75.99,69,90,0.94,overcast clouds
20,20,Bandarbeyla,SO,9.49,50.81,77.77,77,0,20.80,clear sky
22,22,Boffa,GN,10.17,-14.03,82.63,77,18,8.12,few clouds
30,30,Tessalit,ML,20.20,1.01,82.38,15,82,7.72,broken clouds
31,31,Atuona,PF,-9.80,-139.03,77.85,79,22,16.20,light rain
32,32,Poya,NC,-21.35,165.15,72.00,80,0,6.58,clear sky
34,34,Bosaso,SO,11.28,49.18,79.27,71,9,8.68,clear sky


In [16]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [17]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
7,7,Kutum,SD,14.20,24.67,80.11,21,53,9.44,broken clouds
9,9,Rikitea,PF,-23.12,-134.97,74.93,84,42,15.64,light rain
11,11,San Lorenzo,PY,-25.33,-57.53,87.80,27,20,6.93,few clouds
15,15,Hilo,US,19.73,-155.09,75.99,69,90,0.94,overcast clouds
20,20,Bandarbeyla,SO,9.49,50.81,77.77,77,0,20.80,clear sky
22,22,Boffa,GN,10.17,-14.03,82.63,77,18,8.12,few clouds
30,30,Tessalit,ML,20.20,1.01,82.38,15,82,7.72,broken clouds
31,31,Atuona,PF,-9.80,-139.03,77.85,79,22,16.20,light rain
32,32,Poya,NC,-21.35,165.15,72.00,80,0,6.58,clear sky
34,34,Bosaso,SO,11.28,49.18,79.27,71,9,8.68,clear sky


In [18]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Kutum,SD,80.11,broken clouds,14.20,24.67,
9,Rikitea,PF,74.93,light rain,-23.12,-134.97,
11,San Lorenzo,PY,87.80,few clouds,-25.33,-57.53,
15,Hilo,US,75.99,overcast clouds,19.73,-155.09,
20,Bandarbeyla,SO,77.77,clear sky,9.49,50.81,
22,Boffa,GN,82.63,few clouds,10.17,-14.03,
30,Tessalit,ML,82.38,broken clouds,20.20,1.01,
31,Atuona,PF,77.85,light rain,-9.80,-139.03,
32,Poya,NC,72.00,clear sky,-21.35,165.15,
34,Bosaso,SO,79.27,clear sky,11.28,49.18,


In [19]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [26]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Kutum,SD,80.11,broken clouds,14.20,24.67,Dibbo areaمنطقة دبو
9,Rikitea,PF,74.93,light rain,-23.12,-134.97,Pension Maro'i
11,San Lorenzo,PY,87.80,few clouds,-25.33,-57.53,Ámbar Hotel
15,Hilo,US,75.99,overcast clouds,19.73,-155.09,Hilo Hawaiian Hotel
20,Bandarbeyla,SO,77.77,clear sky,9.49,50.81,REER XOOSHLE GROUP
...,...,...,...,...,...,...,...
700,Envira,BR,88.16,overcast clouds,-7.30,-70.22,
705,Puerto Del Rosario,ES,71.60,few clouds,28.50,-13.86,Hotel JM Puerto del Rosario
707,Trairi,BR,86.56,few clouds,-3.28,-39.27,Pousada Field Lilies - Trairi
714,Vao,NC,74.05,broken clouds,-22.67,167.48,Hotel Kou-Bugny


In [27]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [28]:
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Kutum,SD,80.11,broken clouds,14.20,24.67,Dibbo areaمنطقة دبو
9,Rikitea,PF,74.93,light rain,-23.12,-134.97,Pension Maro'i
11,San Lorenzo,PY,87.80,few clouds,-25.33,-57.53,Ámbar Hotel
15,Hilo,US,75.99,overcast clouds,19.73,-155.09,Hilo Hawaiian Hotel
20,Bandarbeyla,SO,77.77,clear sky,9.49,50.81,REER XOOSHLE GROUP
...,...,...,...,...,...,...,...
700,Envira,BR,88.16,overcast clouds,-7.30,-70.22,
705,Puerto Del Rosario,ES,71.60,few clouds,28.50,-13.86,Hotel JM Puerto del Rosario
707,Trairi,BR,86.56,few clouds,-3.28,-39.27,Pousada Field Lilies - Trairi
714,Vao,NC,74.05,broken clouds,-22.67,167.48,Hotel Kou-Bugny


In [32]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [33]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))